In [1]:

# Python 3.12-compatible dependencies for Google Colab (Oct 2025)
# Note: Colab has TensorFlow 2.19+ pre-installed, so we don't downgrade it
!pip install -q torch==2.4.0+cu121 torchvision==0.19.0+cu121 torchaudio==2.4.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -q vit-keras
!pip install -q scikit-learn
!pip install -q torch-geometric -f https://data.pyg.org/whl/torch-2.4.0+cu121.html



In [2]:
!pip install -q tensorflow


In [3]:
# Python 3.12-compatible dependencies for Google Colab
!pip install -q torch==2.4.0+cu121 torchvision==0.19.0+cu121 torchaudio==2.4.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -q "tensorflow>=2.16,<2.18"
!pip install -q vit-keras
!pip install -q scikit-learn
!pip install -q torch-geometric -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

In [6]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import time
import gc

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)

print("Importing TensorFlow and Keras...")
import tensorflow as tf
tf.random.set_seed(42)
print(f'TensorFlow version: {tf.__version__}')

# Configure GPU memory growth to avoid OOM
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f'GPU Available: {len(gpus)} GPU(s) configured with memory growth')
except Exception as e:
    print(f'GPU configuration note: {e}')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, matthews_corrcoef
)
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Use Hugging Face Vision Transformer (compatible with TF 2.19+)
from transformers import TFViTModel, ViTImageProcessor

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Set torch device and seed
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    TORCH_DEVICE = torch.device('cuda')
    print(f'PyTorch using CUDA: {torch.cuda.get_device_name(0)}')
else:
    TORCH_DEVICE = torch.device('cpu')
    print('PyTorch using CPU')

print('✓ All imports successful')






Importing TensorFlow and Keras...
TensorFlow version: 2.17.1
PyTorch using CPU
✓ All imports successful


In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
train_path = '/content/drive/MyDrive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/train'
test_path = '/content/drive/MyDrive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/valid'


if not os.path.exists(train_path):
    print(f"Warning: Training path {train_path} does not exist.")
    print("Please update the paths or upload your dataset to Google Colab.")


if not os.path.exists(test_path):
    print(f"Warning: Test path {test_path} does not exist.")
    print("Please update the paths or upload your dataset to Google Colab.")

def display_sample_images(path):
    if os.path.exists(path):
        disease_list = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        if len(disease_list) > 0:
            fig, axes = plt.subplots(2, 5, figsize=(15, 8))
            fig.suptitle('Sample Images of Leaf Diseases', fontsize=16)
            for i, disease in enumerate(disease_list[:10]):
                disease_path = os.path.join(path, disease)
                if os.path.exists(disease_path):
                    images = [f for f in os.listdir(disease_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    if images:
                        sample_image = random.choice(images)
                        try:
                            img = tf.keras.utils.load_img(os.path.join(disease_path, sample_image))
                        except AttributeError:
                            img = tf.keras.preprocessing.image.load_img(os.path.join(disease_path, sample_image))
                        row, col = i // 5, i % 5
                        if row < 2 and col < 5:
                            axes[row, col].imshow(img)
                            axes[row, col].set_title(disease, fontsize=12)
                            axes[row, col].axis('off')
            for i in range(len(disease_list), 10):
                row, col = i // 5, i % 5
                if row < 2 and col < 5:
                    axes[row, col].axis('off')

            plt.tight_layout()
            plt.show()
    else:
        print(f"Path {path} not found. Skipping image display.")

MessageError: Error: credential propagation was unsuccessful